In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [44]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df["Revenue"] = df["Total Environmental Cost"]/df["Environmental Intensity (Sales)"]
df["Operating Income"] = df["Total Environmental Cost"]/df["Environmental Intensity (Op Inc)"]
df["Environmental Intensity (Op Inc)"] = df["Environmental Intensity (Op Inc)"]*100
df["Environmental Intensity (Sales)"] = df["Environmental Intensity (Sales)"]*100


df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,244464000.0
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,56797000.0
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,638704000.0
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.462497,NaN,-2.290124e+06,-2.239814e+06,-510.210093,...,-0.189720,-1.009642,-253.366805,-0.226133,-3.169102,-3.169102,-92.619013,0.016390,1.565900e+08,NaN
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,784560000.0


In [31]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

independentVariables = sdg_set
dependentVariables = "Environmental Intensity (Sales)"

In [45]:
Q1 = df['Environmental Intensity (Op Inc)'].quantile(0.25)
Q3 = df['Environmental Intensity (Op Inc)'].quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers based on the IQR method
df_cleaned = df[(df['Environmental Intensity (Op Inc)'] >= lower_bound) & (df['Environmental Intensity (Op Inc)'] <= upper_bound)]

df_no_outliers
#df_cleaned = df[df['% Imputed'] <= 0.2 * df['% Imputed'].max()]

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,2.444640e+08
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,5.679700e+07
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,6.387040e+08
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,7.845600e+08
5,2019,RICARDO PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.665664,-7.274924,-3.247235e+06,-3.176408e+06,-753.133044,...,-2.096319,-0.454339,-373.412363,-2.498664,31.257053,31.257053,-132.536045,0.025666,4.878190e+08,4.463600e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14020,2010,TOKYU CORP,JAPAN,Railroads,Transport via railways,-1.711919,-37.223571,-2.596747e+08,-2.543014e+08,-67355.726636,...,-1340.008870,-803.769152,-30891.641257,-1597.196181,11617.281453,11617.281453,-10046.369938,0.204184,1.516863e+10,6.976082e+08
14021,2010,KINTETSU GROUP HOLDINGS CO,JAPAN,Railroads,Transport via railways,-1.389543,-36.983489,-1.646121e+08,-1.612440e+08,-44059.705138,...,-1046.528309,-627.732540,-19853.851852,-1247.388025,9072.935399,9072.935399,-6291.028013,0.237866,1.184649e+10,4.450961e+08
14022,2010,UNITED NATURAL FOODS INC,UNITED STATES OF AMERICA,Food Distributors,"Wholesale trade and commission trade, except o...",-1.014760,-33.181267,-3.812594e+07,-3.714697e+07,-8999.801240,...,-54.712627,-7.953707,-4423.435378,-65.213597,752.525332,752.525332,-1547.280723,0.047708,3.757139e+09,1.149020e+08
14023,2010,SPAR GROUP LTD,SOUTH AFRICA,Food Distributors,"Wholesale trade and commission trade, except o...",-0.415410,-14.650443,-2.186324e+07,-2.138385e+07,-5415.923810,...,-72.617455,-44.562633,-2555.464157,-86.554891,811.816131,811.816131,-871.304107,0.138048,5.263047e+09,1.492326e+08


In [51]:

df_cleaned = df_no_outliers[df_no_outliers['% Imputed'] <= 0.25 * df_no_outliers['% Imputed'].max()]
df_cleaned

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,2.444640e+08
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,5.679700e+07
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,6.387040e+08
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,7.845600e+08
5,2019,RICARDO PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.665664,-7.274924,-3.247235e+06,-3.176408e+06,-753.133044,...,-2.096319,-0.454339,-373.412363,-2.498664,31.257053,31.257053,-132.536045,0.025666,4.878190e+08,4.463600e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13998,2010,COSTCO WHOLESALE CORP,UNITED STATES OF AMERICA,Hypermarkets & Super Centers,"Retail trade, except of motor vehicles and mot...",-0.601445,-22.549425,-4.688026e+08,-4.585981e+08,-112939.889978,...,-873.713966,-126.361293,-55249.839254,-1041.405502,12019.103529,12019.103529,-19106.517600,0.060416,7.794600e+10,2.079000e+09
14006,2010,WAL MART DE MEXICO SA,MEXICO,Hypermarkets & Super Centers,"Retail trade, except of motor vehicles and mot...",0.068825,0.895888,1.866767e+07,3.673800e+07,-151926.588443,...,-16741.590078,-1.672746,-56213.731387,-19954.796074,247421.293148,247421.293148,-289.588052,0.000030,2.712346e+10,2.083707e+09
14008,2010,HOTAI MOTOR CO,"TAIWAN, PROVINCE OF CHINA",Automotive Retail,"Sale, maintenance, repair of motor vehicles, m...",-2.877303,-68.912903,-1.093154e+08,-1.069441e+08,-24678.818094,...,-26.962315,-12.493538,-12288.866128,-32.137180,349.755455,349.755455,-4448.329839,0.010909,3.799232e+09,1.586284e+08
14014,2010,VOPAK (KONINKLIJKE) NV,NETHERLANDS,Oil & Gas Storage & Transportation,Transport via pipelines,-8.394585,-26.263940,-1.246136e+08,-1.219107e+08,-27940.705644,...,-7.682563,-3.125430,-13956.602935,-9.157074,94.842606,94.842606,-5073.060652,0.002689,1.484452e+09,4.744663e+08


In [50]:
# Create dummy variables for fixed effects on Year and GICS Sub-Industry
year_dummies = pd.get_dummies(df_cleaned['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)

# Define your dependent variable
y = df_cleaned[dependentVariables]

# Define your independent variables (including Year and Sub-Industry dummies)
X = df_cleaned[independentVariables]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([year_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year): {adjusted_r_squared:.6f}')

Adjusted R-squared (Fixed on Year): 0.000044


In [47]:
# Create dummy variables for fixed effects on Year and GICS Sub-Industry
year_dummies = pd.get_dummies(df_cleaned['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)
subindustry_dummies = pd.get_dummies(df_cleaned['GICS Sub-Industry'], drop_first=True)
subindustry_dummies = subindustry_dummies.astype(int)

# Define your dependent variable
y = df_cleaned[dependentVariables]

# Define your independent variables (including Year and Sub-Industry dummies)
X = df_cleaned[independentVariables]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([year_dummies, subindustry_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year and GICS Sub-Industry): {adjusted_r_squared:.4f}')


Adjusted R-squared (Fixed on Year and GICS Sub-Industry): 0.3995


In [48]:
# Create dummy variables for fixed effects on Year and GICS Sub-Industry
year_dummies = pd.get_dummies(df_cleaned['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)
Industry_dummies = pd.get_dummies(df_cleaned['Industry (Exiobase)'], drop_first=True)
Industry_dummies = Industry_dummies.astype(int)

# Define your dependent variable
y = df_cleaned[dependentVariables]

# Define your independent variables (including Year and Sub-Industry dummies)
X = df_cleaned[independentVariables]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([year_dummies, Industry_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')


Adjusted R-squared (Fixed on Year and Industry (Exiobase)): 0.3108


In [28]:

# Define your dependent variable
y = df_cleaned['Environmental Intensity (Sales)']

# Define your independent variables (including Year and Sub-Industry dummies)
X = df_cleaned[['Total Environmental Cost', 'Working Capacity', 'Fish Production Capacity', 'Crop Production Capacity', 'Meat Production Capacity', 'Biodiversity', 'Abiotic Resources', 'Water production capacity (Drinking water & Irrigation Water)', 'Wood Production Capacity', 'SDG 1.5', 'SDG 2.1', 'SDG 2.2', 'SDG 2.3', 'SDG 2.4', 'SDG 3.3', 'SDG 3.4', 'SDG 3.9', 'SDG 6', 'SDG 12.2', 'SDG 14.1', 'SDG 14.2', 'SDG 14.3', 'SDG 14.c', 'SDG 15.1', 'SDG 15.2', 'SDG 15.5']]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([X], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')


Adjusted R-squared (Fixed on Year and Industry (Exiobase)): 0.1127
